In [5]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns

from sklearn import datasets 
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer

In [100]:
# read lat/lng data into panda dataframe
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')

# df = pd.read_sql_query("SELECT latitude, longitude FROM fires, STATE", conn)
df = pd.read_sql_query("SELECT FIRE_YEAR, STAT_CAUSE_DESCR, LATITUDE, LONGITUDE, STATE, COUNTY, FIPS_NAME, DISCOVERY_DATE, FIRE_SIZE, DISCOVERY_DOY, DISCOVERY_TIME FROM 'Fires'", conn)
df['DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

FIRE_YEAR STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE COUNTY  FIPS_NAME  \
0       2005    Miscellaneous  40.036944 -121.005833    CA     63     Plumas   
1       2004        Lightning  38.933056 -120.404444    CA     61     Placer   
2       2004   Debris Burning  38.984167 -120.735556    CA     17  El Dorado   
3       2004        Lightning  38.559167 -119.913333    CA      3     Alpine   
4       2004        Lightning  38.559167 -119.933056    CA      3     Alpine   

   DISCOVERY_DATE  FIRE_SIZE  DISCOVERY_DOY DISCOVERY_TIME       DATE  
0       2453403.5       0.10             33           1300 2005-02-02  
1       2453137.5       0.25            133           0845 2004-05-12  
2       2453156.5       0.10            152           1921 2004-05-31  
3       2453184.5       0.10            180           1600 2004-06-28  
4       2453184.5       0.10            180           1600 2004-06-28  


In [101]:
df['TIME'] = df['DISCOVERY_TIME'].str[0:2]
df['DATETIME'] = df['DATE'].dt.strftime('%Y-%m-%d') + ' ' +  df['TIME']
print(df)

FIRE_YEAR   STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE COUNTY  \
0             2005      Miscellaneous  40.036944 -121.005833    CA     63   
1             2004          Lightning  38.933056 -120.404444    CA     61   
2             2004     Debris Burning  38.984167 -120.735556    CA     17   
3             2004          Lightning  38.559167 -119.913333    CA      3   
4             2004          Lightning  38.559167 -119.933056    CA      3   
...            ...                ...        ...         ...   ...    ...   
1880460       2015  Missing/Undefined  40.481637 -122.389375    CA   None   
1880461       2015      Miscellaneous  37.617619 -120.938570    CA   None   
1880462       2015  Missing/Undefined  37.617619 -120.938570    CA   None   
1880463       2015  Missing/Undefined  37.672235 -120.898356    CA   None   
1880464       2015      Miscellaneous  34.263217 -116.830950    CA   None   

         FIPS_NAME  DISCOVERY_DATE  FIRE_SIZE  DISCOVERY_DOY DISCOVERY_TIME  \
0    

In [102]:
df = df[df['FIRE_YEAR'] > 2012]
print(df)

FIRE_YEAR   STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE COUNTY  \
1635454       2013      Miscellaneous  40.391944 -123.061944    CA    105   
1635455       2013      Miscellaneous  38.858056 -120.762222    CA    017   
1635456       2013          Lightning  45.786667 -114.386944    MT    081   
1635457       2013          Lightning  39.152222 -120.233333    CA    061   
1635458       2013          Lightning  37.691667 -108.621944    CO    083   
...            ...                ...        ...         ...   ...    ...   
1880460       2015  Missing/Undefined  40.481637 -122.389375    CA   None   
1880461       2015      Miscellaneous  37.617619 -120.938570    CA   None   
1880462       2015  Missing/Undefined  37.617619 -120.938570    CA   None   
1880463       2015  Missing/Undefined  37.672235 -120.898356    CA   None   
1880464       2015      Miscellaneous  34.263217 -116.830950    CA   None   

         FIPS_NAME  DISCOVERY_DATE  FIRE_SIZE  DISCOVERY_DOY DISCOVERY_TIME  \
16354

In [7]:
# df_fire = pd.read_csv('California_Fire_Incidents.csv')
df_humidity = pd.read_csv('humidity.csv')
df_pressure = pd.read_csv('pressure.csv')
df_temp = pd.read_csv('temperature.csv')
df_weather = pd.read_csv('weather_description.csv')
df_wind_direction = pd.read_csv('wind_direction.csv')
df_wind_speed = pd.read_csv('wind_speed.csv')
df_city_attributes = pd.read_csv('city_attributes.csv')

In [8]:
cities = df_humidity.columns.drop(['Jerusalem', 'Haifa', 'Eilat', 'Tel Aviv District', 'Beersheba', 'Nahariyya'])
df_humidity = df_humidity[cities]
df_pressure = df_pressure[cities]
df_temp = df_temp[cities]
df_weather = df_weather[cities]
df_wind_direction = df_wind_direction[cities]
df_wind_speed = df_wind_speed[cities]
# df_city_attributes = df_city_attributes[cities]

In [131]:
print(df_humidity)

datetime  Vancouver  Portland  San Francisco  Seattle  \
0      2012-10-01 13:00:00       76.0      81.0           88.0     81.0   
1      2012-10-01 14:00:00       76.0      80.0           87.0     80.0   
2      2012-10-01 15:00:00       76.0      80.0           86.0     80.0   
3      2012-10-01 16:00:00       77.0      80.0           85.0     79.0   
4      2012-10-01 17:00:00       78.0      79.0           84.0     79.0   
...                    ...        ...       ...            ...      ...   
45247  2017-11-29 20:00:00        NaN      81.0            NaN     93.0   
45248  2017-11-29 21:00:00        NaN      71.0            NaN     87.0   
45249  2017-11-29 22:00:00        NaN      71.0            NaN     93.0   
45250  2017-11-29 23:00:00        NaN      71.0            NaN     87.0   
45251  2017-11-30 00:00:00        NaN      76.0            NaN     75.0   

       Los Angeles  San Diego  Las Vegas  Phoenix  Albuquerque  ...  Detroit  \
0             88.0       82.0       2

In [217]:
common_cities = []
for city in cities:
    if(len(df.loc[df['FIPS_NAME']==city]) != 0):
        common_cities.append(city)

In [218]:
df = df[df['FIPS_NAME'].isin(common_cities)]

In [192]:
columns = ['time', 'humidity', 'pressure', 'temperature', 'weather', 'wind direction', 'wind speed']
df_weatherdata = pd.DataFrame()
for city in common_cities:
    newdf = pd.DataFrame(pd.concat([df_humidity['datetime'], df_humidity[city], df_pressure[city], df_temp[city], df_weather[city], df_wind_direction[city], df_wind_speed[city]], axis=1, keys=columns))
    newdf['city'] = city
    df_weatherdata = df_weatherdata.append(newdf, ignore_index=True)

In [193]:
print(df_weatherdata['city'])

0         San Francisco
1         San Francisco
2         San Francisco
3         San Francisco
4         San Francisco
              ...      
407263         New York
407264         New York
407265         New York
407266         New York
407267         New York
Name: city, Length: 407268, dtype: object


In [194]:
df_weatherdata['DATE'] = df_weatherdata['time'].str[0:11]
df_weatherdata['TIME'] = df_weatherdata['time'].str.split(":").str[0].str[11:]
df_weatherdata['DATETIME'] = df_weatherdata['DATE'] + df_weatherdata['TIME']
print(df_weatherdata['DATETIME'])

0         2012-10-01 13
1         2012-10-01 14
2         2012-10-01 15
3         2012-10-01 16
4         2012-10-01 17
              ...      
407263    2017-11-29 20
407264    2017-11-29 21
407265    2017-11-29 22
407266    2017-11-29 23
407267    2017-11-30 00
Name: DATETIME, Length: 407268, dtype: object


In [214]:
df_weatherdata['DATETIME_CITY'] = df_weatherdata['DATETIME'] + ' ' + df_weatherdata['city']
print(df_weatherdata['DATETIME_CITY'])

0         2012-10-01 13 San Francisco
1         2012-10-01 14 San Francisco
2         2012-10-01 15 San Francisco
3         2012-10-01 16 San Francisco
4         2012-10-01 17 San Francisco
                     ...             
407263         2017-11-29 20 New York
407264         2017-11-29 21 New York
407265         2017-11-29 22 New York
407266         2017-11-29 23 New York
407267         2017-11-30 00 New York
Name: DATETIME_CITY, Length: 407268, dtype: object


In [219]:
df['DATETIME_CITY'] = df['DATETIME'] + ' ' + df['FIPS_NAME']
print(df['DATETIME_CITY'])

1635479    2013-08-14 14 Los Angeles
1635500      2013-02-14 16 San Diego
1635530    2013-07-30 13 Los Angeles
1635568    2013-06-02 21 Los Angeles
1635576      2013-09-08 23 San Diego
                     ...            
1872185      2015-11-28 15 San Diego
1872229      2015-12-29 17 San Diego
1872237    2015-11-04 14 Los Angeles
1872244      2015-10-13 03 San Diego
1872247      2015-12-09 16 San Diego
Name: DATETIME_CITY, Length: 2933, dtype: object


In [195]:
if df_weatherdata['DATETIME'][0] in (df['DATETIME']):
    print('true')
else:
    print('false')
# print(df['DATETIME'])

false


In [211]:
print(df_weatherdata['DATETIME'][1+7340])
print(df['DATETIME'].values[2])
if df_weatherdata['DATETIME'][1+7340] in (df['DATETIME'].values):
    print(np.where(df['DATETIME'].values == (df_weatherdata['DATETIME'][1+7340])))

2013-08-03 10
2013-08-03 10
(array([    2,  3519,  5633,  6084,  9170,  9794, 14878, 19667, 23423,
       33748]),)


In [224]:
df_weatherdata['FIRE'] = False

In [227]:
df_weatherdata['FIRE'][df_weatherdata.loc[df_weatherdata['DATETIME_CITY'].isin(df['DATETIME_CITY'].values)].index] = True

In [231]:
print(df_weatherdata.loc[df_weatherdata['FIRE'] == True])

time  humidity  pressure  temperature  \
14176   2014-05-15 05:00:00      69.0    1017.0   294.270000   
14221   2014-05-17 02:00:00       NaN    1014.0   292.180000   
20087   2015-01-16 12:00:00     100.0    1038.0   282.415000   
21515   2015-03-17 00:00:00      77.0    1030.0   289.181500   
24145   2015-07-04 14:00:00      87.0    1012.0   289.090000   
...                     ...       ...       ...          ...   
389506  2015-11-20 23:00:00      45.0    1025.0   282.632909   
389739  2015-11-30 16:00:00      60.0    1030.0   276.642214   
389810  2015-12-03 15:00:00      66.0    1011.0   281.390000   
389859  2015-12-05 16:00:00      65.0    1039.0   280.294636   
390247  2015-12-21 20:00:00      53.0    1025.0   283.583269   

                     weather  wind direction  wind speed           city  \
14176           sky is clear           271.0         2.0  San Francisco   
14221                   haze           334.0         1.0  San Francisco   
20087          broken clouds 